In [21]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras._tf_keras.keras.utils import to_categorical

In [22]:
IMAGE_SIZE = (160, 160) 
BATCH_SIZE = 32
EPOCHS = 28
TRAIN_DIR = "train" 
VALIDATION_SPLIT = 0.3 

In [23]:
try:
    categories = sorted(os.listdir(TRAIN_DIR))  #Listed Categories
    if not categories:
        raise ValueError("Training directory is empty.")
except FileNotFoundError:
    raise FileNotFoundError(f"Training directory '{TRAIN_DIR}' does not exist.")

In [24]:
# Function to load in OpenCV instead of Pillow
def load_data_with_cv2(directory, categories, image_size):
    images = []
    labels = []
    for idx, category in enumerate(categories):
        category_path = os.path.join(directory, category)
        if not os.path.isdir(category_path):
            print(f"Warning: '{category_path}' is not a directory. Skipping.")
            continue
        for file in os.listdir(category_path):
            img_path = os.path.join(category_path, file)
            try:
                img = cv2.imread(img_path)  # Load image
                if img is None:
                    print(f"Warning: Could not load image '{img_path}'. Skipping.")
                    continue
                img = cv2.resize(img, image_size)  # Resize to Size set in kernal 2
                img = img / 255.0  # pixel values to [0, 1]
                images.append(img)
                labels.append(idx)
            except Exception as e:
                print(f"Error loading image '{img_path}': {e}")
    if not images:
        raise ValueError("No valid images found in the dataset.")
    return np.array(images), np.array(labels)

In [25]:
# Load training
train_images, train_labels = load_data_with_cv2(TRAIN_DIR, categories, IMAGE_SIZE)

# Shuffle and split the data manually using NumPy
indices = np.arange(len(train_images))
np.random.seed(42)  # Ensure reproducibility
np.random.shuffle(indices)

# Compute the split index
split_idx = int(len(train_images) * (1 - VALIDATION_SPLIT))

# Split the data
train_indices, val_indices = indices[:split_idx], indices[split_idx:]
train_images, val_images = train_images[train_indices], train_images[val_indices]
train_labels, val_labels = train_labels[train_indices], train_labels[val_indices]

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, num_classes=len(categories))
val_labels = to_categorical(val_labels, num_classes=len(categories))

# Create TensorFlow datasets for training and validation
train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data = train_data.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_data = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_data = val_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [26]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')  # One output per category
])

In [27]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 158, 158, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 79, 79, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 79, 79, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 77, 77, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 38, 38, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 38, 38, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 36, 36, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,402,308 (20.61 MB)

 Trainable params: 5,402,308 (20.61 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - accuracy: 0.2689 - loss: 3.6493 - val_accuracy: 0.2561 - val_loss: 1.3847
Epoch 2/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 280ms/step - accuracy: 0.3164 - loss: 1.3814 - val_accuracy: 0.3049 - val_loss: 1.3815
Epoch 3/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - accuracy: 0.2727 - loss: 1.3875 - val_accuracy: 0.2439 - val_loss: 1.3835
Epoch 4/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 295ms/step - accuracy: 0.3375 - loss: 1.3697 - val_accuracy: 0.3049 - val_loss: 1.3828
Epoch 5/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 301ms/step - accuracy: 0.3213 - loss: 1.3572 - val_accuracy: 0.3049 - val_loss: 1.3764
Epoch 6/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 283ms/step - accuracy: 0.3202 - loss: 1.3494 - val_accuracy: 0.3537 - val_loss: 1.3826
Epoch 7/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 264ms/step - accuracy: 0.3723 - loss: 1.3410 - val_accuracy: 0.3659 - val_loss: 1.3690
Epoch 8/28
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 270ms/step - accuracy: 0.3227 - loss: 1.3627 - val_accuracy: 0.3293 - val_loss:

In [30]:
# Save the trained model
model.save("cat_emotion_model.h5")

print("Model training complete. Saved as 'cat_emotion_model.h5'.")

Model training complete. Saved as 'cat_emotion_model.h5'.
